# Generating an interactive climate map in Folium (& Geopandas) for Berlin¶

In [2]:
import pandas as pd
import BerlinTempMap_modules as btm
import folium
import re
import random
import numpy as np
import math

#### Get temperature data fitting following list:

['Reinickendorf',
 'Charlottenburg-Wilmersdorf',
 'Treptow-Köpenick',
 'Pankow',
 'Neukölln',
 'Lichtenberg',
 'Marzahn-Hellersdorf',
 'Spandau',
 'Steglitz-Zehlendorf',
 'Mitte',
 'Friedrichshain-Kreuzberg',
 'Tempelhof-Schöneberg']

Reinickendorf --> BERLIN-TEGEL

Charlottenburg-Wilmersdorf --> BERLIN-DAHLEM

Treptow-Köpenick --> BERLIN-TREPTOW

Pankow --> BERLIN-BUCH

Neukölln --> BERLIN-RUDOW (or BERLIN-SCHOENEFELD; Rudow chosen for now)

Lichtenberg --> BERLIN-INVALIDENSTRASSE

Marzahn-Hellersdorf --> BERLIN-MARZAHN

Spandau --> BERLIN-SPANDAU

Steglitz-Zehlendorf --> BERLIN-ZEHLENDORF

Mitte --> BERLIN-MITTE

Friedrichshain-Kreuzberg --> BERLIN-OSTKREUZ

Tempelhof-Schöneberg --> BERLIN-TEMPELHOF

## Read in temperature data for all the districts of Berlin

In [3]:
df_reinickendorf = btm.read_in('data/TG_STAID004005_tegel_prepared.csv', 'Reinickendorf')
df_charlottenburg_wilmersdorf = btm.read_in('data/TG_STAID000041_dahlem_prepared.csv', 'Charlottenburg-Wilmersdorf')
df_treptow_koepenick = btm.read_in('data/TG_STAID004586_treptow_prepared.csv', 'Treptow-Köpenick')
df_pankow = btm.read_in('data/TG_STAID004529_buch_prepared.csv', 'Pankow')
df_neukoelln = btm.read_in('data/TG_STAID004566_rudow_prepared.csv', 'Neukölln')
df_lichtenberg = btm.read_in('data/TG_STAID011739_invalidenstr_prepared.csv', 'Lichtenberg')
df_marzahn_hellersdorf = btm.read_in('data/TG_STAID004561_marzahn_prepared.csv', 'Marzahn-Hellersdorf')
df_spandau = btm.read_in('data/TG_STAID004575_spandau_prepared.csv', 'Spandau')
df_steglitz_zehlendorf = btm.read_in('data/TG_STAID004588_zehlendorf_prepared.csv', 'Steglitz-Zehlendorf')
df_mitte = btm.read_in('data/TG_STAID004563_mitte_prepared.csv', 'Mitte')
df_friedrichshain_kreuzberg = btm.read_in('data/TG_STAID011740_ostkreuz_prepared.csv', 'Friedrichshain-Kreuzberg')
df_tempelhof_schoeneberg = btm.read_in('data/TG_STAID002759_tempelhof_prepared.csv', 'Tempelhof-Schöneberg')
df_tempelhof_schoeneberg.head()

,district,daily mean temp,Q_TG,month,year
DATE,,,,,
1876-01-01,Tempelhof-Schöneberg,2.2,0,1,1876
1876-01-02,Tempelhof-Schöneberg,2.5,0,1,1876
1876-01-03,Tempelhof-Schöneberg,0.3,0,1,1876
1876-01-04,Tempelhof-Schöneberg,-5.8,0,1,1876
1876-01-05,Tempelhof-Schöneberg,-9.8,0,1,1876


In [4]:
all_districts = [df_reinickendorf, df_charlottenburg_wilmersdorf, df_treptow_koepenick, df_pankow, df_neukoelln, df_lichtenberg, df_marzahn_hellersdorf, df_spandau, df_steglitz_zehlendorf, df_mitte, df_friedrichshain_kreuzberg, df_tempelhof_schoeneberg]
for x in all_districts:
    print(x.head(1))

                 district  daily mean temp  Q_TG  month  year
    DATE                                                     
1876-01-01  Reinickendorf              2.2     0      1  1876
                              district  daily mean temp  Q_TG  month  year
    DATE                                                                  
1876-01-01  Charlottenburg-Wilmersdorf              2.2     0      1  1876
                    district  daily mean temp  Q_TG  month  year
    DATE                                                        
1876-01-01  Treptow-Köpenick              2.2     0      1  1876
           district  daily mean temp  Q_TG  month  year
    DATE                                               
1961-01-01   Pankow              1.2     0      1  1961
            district  daily mean temp  Q_TG  month  year
    DATE                                                
1936-01-01  Neukölln              2.4     0      1  1936
               district  daily mean temp  Q_TG  month  

**Spandau starts at 1963-01-01. So complete set only after 1963-01-01**

In [5]:
for x in all_districts:
    print(x.tail(1))

                 district  daily mean temp  Q_TG  month  year
    DATE                                                     
2020-10-31  Reinickendorf             13.9     0     10  2020
                              district  daily mean temp  Q_TG  month  year
    DATE                                                                  
2020-10-31  Charlottenburg-Wilmersdorf             13.6     0     10  2020
                    district  daily mean temp  Q_TG  month  year
    DATE                                                        
2020-10-31  Treptow-Köpenick             13.9     0     10  2020
           district  daily mean temp  Q_TG  month  year
    DATE                                               
2020-10-31   Pankow             13.8     0     10  2020
            district  daily mean temp  Q_TG  month  year
    DATE                                                
2020-10-31  Neukölln             13.8     0     10  2020
               district  daily mean temp  Q_TG  month  

In [6]:
df_reinickendorf = btm.addmonthlymean(df_reinickendorf)
df_charlottenburg_wilmersdorf = btm.addmonthlymean(df_charlottenburg_wilmersdorf)
df_treptow_koepenick = btm.addmonthlymean(df_treptow_koepenick)
df_pankow = btm.addmonthlymean(df_pankow)
df_neukoelln = btm.addmonthlymean(df_neukoelln)
df_lichtenberg = btm.addmonthlymean(df_lichtenberg)
df_marzahn_hellersdorf = btm.addmonthlymean(df_marzahn_hellersdorf)
df_spandau = btm.addmonthlymean(df_spandau)
df_steglitz_zehlendorf = btm.addmonthlymean(df_steglitz_zehlendorf)
df_mitte = btm.addmonthlymean(df_mitte)
df_friedrichshain_kreuzberg = btm.addmonthlymean(df_friedrichshain_kreuzberg)
df_tempelhof_schoeneberg = btm.addmonthlymean(df_tempelhof_schoeneberg)
df_mitte.head()

,district,daily mean temp,Q_TG,month,year,monthly mean temp
DATE,,,,,,
1876-01-01,Mitte,2.2,0,1,1876,-2.070968
1876-01-02,Mitte,2.5,0,1,1876,-2.070968
1876-01-03,Mitte,0.3,0,1,1876,-2.070968
1876-01-04,Mitte,-5.8,0,1,1876,-2.070968
1876-01-05,Mitte,-9.8,0,1,1876,-2.070968


**combine the 12 data frames into one common data frame**

In [7]:
df_full = pd.concat([df_reinickendorf, df_charlottenburg_wilmersdorf, df_treptow_koepenick, df_pankow, df_neukoelln, df_lichtenberg, df_marzahn_hellersdorf, df_spandau, df_steglitz_zehlendorf, df_mitte, df_friedrichshain_kreuzberg, df_tempelhof_schoeneberg])
df_full.to_csv('data/df_full.csv')
df_full.head(2)

,district,daily mean temp,Q_TG,month,year,monthly mean temp
DATE,,,,,,
1876-01-01,Reinickendorf,2.2,0,1,1876,-2.070968
1876-01-02,Reinickendorf,2.5,0,1,1876,-2.070968


In [8]:
df_full.tail(2)

,district,daily mean temp,Q_TG,month,year,monthly mean temp
DATE,,,,,,
2020-10-30,Tempelhof-Schöneberg,11.4,0,10,2020,11.516129
2020-10-31,Tempelhof-Schöneberg,13.9,0,10,2020,11.516129


## Read in the geographic data (geometric shapes of all Berlin districts)

In [9]:
world_map=folium.Map()
world_map

In [10]:
berlin = folium.Map(location=[52.382, 13.41],
                zoom_start=12,
                width=600, height=600,
                control_scale=True,
                tiles='OpenStreetMap')
berlin

In [11]:
berlin.save('data/mapBerlin.html')

In [12]:
geojson = open('data/bezirksgrenzen.geojson').read()
print(geojson[:600])

{
"type": "FeatureCollection",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
                                                                                
"features": [
{ "type": "Feature", "properties": { "gml_id": "s_wfs_alkis_bezirk.F176__1", "Gemeinde_name": "Reinickendorf", "Gemeinde_schluessel": "012", "Land_name": "Berlin", "Land_schluessel": "11", "Schluessel_gesamt": "11000012" }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ 13.320744327762688, 52.626599063597702 ], [ 13.320450024315486, 52.626614320406517 ], [ 13.320156209034


In [13]:
districts = re.findall(r'Gemeinde_name": "([^,]+)",', geojson)
districts

['Reinickendorf',
 'Charlottenburg-Wilmersdorf',
 'Treptow-Köpenick',
 'Pankow',
 'Neukölln',
 'Lichtenberg',
 'Marzahn-Hellersdorf',
 'Spandau',
 'Steglitz-Zehlendorf',
 'Mitte',
 'Friedrichshain-Kreuzberg',
 'Tempelhof-Schöneberg']

In [14]:
value = float(df_full.loc[(df_full['district'] == 'Mitte') & (df_full.index == '2020-10-31')]['monthly mean temp'])
value

11.3258064516129

In [15]:
temps = []
for distr in districts:
    print(distr)
    value = df_full.loc[(df_full['district'] == distr) & (df_full.index == '2020-10-31')]['monthly mean temp']
    temps.append(value)
temps

Reinickendorf
Charlottenburg-Wilmersdorf
Treptow-Köpenick
Pankow
Neukölln
Lichtenberg
Marzahn-Hellersdorf
Spandau
Steglitz-Zehlendorf
Mitte
Friedrichshain-Kreuzberg
Tempelhof-Schöneberg


[    DATE
 2020-10-31    11.325806
 Name: monthly mean temp, dtype: float64,
     DATE
 2020-10-31    10.948387
 Name: monthly mean temp, dtype: float64,
     DATE
 2020-10-31    11.516129
 Name: monthly mean temp, dtype: float64,
     DATE
 2020-10-31    11.045161
 Name: monthly mean temp, dtype: float64,
     DATE
 2020-10-31    11.274194
 Name: monthly mean temp, dtype: float64,
     DATE
 2020-10-31   -999.9
 Name: monthly mean temp, dtype: float64,
     DATE
 2020-10-31    11.367742
 Name: monthly mean temp, dtype: float64,
     DATE
 2020-10-31    11.325806
 Name: monthly mean temp, dtype: float64,
     DATE
 2020-10-31    10.948387
 Name: monthly mean temp, dtype: float64,
     DATE
 2020-10-31    11.325806
 Name: monthly mean temp, dtype: float64,
     DATE
 2020-10-31   -999.9
 Name: monthly mean temp, dtype: float64,
     DATE
 2020-10-31    11.516129
 Name: monthly mean temp, dtype: float64]

In [16]:
temps_float = []
for i in range(len(temps)):
    temps_float.append(temps[i][0])
temps = temps_float

In [17]:
#data = [random.randint(1, 10) for d in districts]

df = pd.DataFrame({'value': temps, 'district': districts})
df

,value,district
0,11.325806,Reinickendorf
1,10.948387,Charlottenburg-Wilmersdorf
2,11.516129,Treptow-Köpenick
3,11.045161,Pankow
4,11.274194,Neukölln
5,-999.900000,Lichtenberg
6,11.367742,Marzahn-Hellersdorf
7,11.325806,Spandau
8,10.948387,Steglitz-Zehlendorf
9,11.325806,Mitte


**Try to spread the small differences in mean temp**

In [18]:
temps_w = df['value'].copy()
for i in range(len(temps_w)):
    if temps_w[i] > 0:
        temps_w[i] = math.exp(temps_w[i])
    else:
        temps_w[i] = 0
df2 = df.copy()
df2['value'] = temps_w
df2

,value,district
0,82934.502738,Reinickendorf
1,56862.258045,Charlottenburg-Wilmersdorf
2,100320.870419,Treptow-Köpenick
3,62640.122753,Pankow
4,78762.600354,Neukölln
5,0.000000,Lichtenberg
6,86486.355058,Marzahn-Hellersdorf
7,82934.502738,Spandau
8,56862.258045,Steglitz-Zehlendorf
9,82934.502738,Mitte


**exp spreads the values to create different colors in the Berlin map.
However, stay with actual mean temp**


## Group / aggregate the mean temperature data by district, year¶

## Merge Data Sets (temperature data and geometric data)

 ## Plot data on a map for a single year (we can make it interactive later)¶

In [20]:
berlin = folium.Map(location=[52.54, 13.36],
                    zoom_start=10,
                    tiles='CartoDB positron')

berlin.choropleth(
    geo_data='data/bezirksgrenzen.geojson',
    name='chloropleth',
    data=df,
    columns=['district', 'value'],
    key_on='properties.Gemeinde_name',
    fill_color='Reds',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Spicy Gradient'
)
folium.LayerControl().add_to(berlin)
berlin

#### Generate a blank canvas / figure.

#### Generate a GeoJSON string for a single year.

## Generate an interactive choropleth map of the data for a single year

## Add additional features to the map!

## Export the figure to an HTML file, so you can open it in your web browser!

####  if you really want to add animation, you either have to write your own custom JavaScript (probably not a reasonable option) or switch over to a more powerful visualization library like Plotly or Bokeh.

possible solution: write script to make a screenshot for each .html produced per year and save it as .png to animate with imageio